In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sample_submission=train_data=pd.read_csv('/kaggle/input/sf-crime/sampleSubmission.csv.zip')
sample_submission.head()

In [ ]:
train_data=train_data=pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
train_data.head()

In [ ]:
test_data=test_data=pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')
test_data.head()

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
list_of_cats=list(train_data['Category'].unique())
list_of_cats

In [ ]:
train_data['year']=train_data['Dates'].apply(lambda x:int(x[0:4]))
train_data['month']=train_data['Dates'].apply(lambda x:int(x[5:7]))
train_data['day']=train_data['Dates'].apply(lambda x:int(x[8:10]))
train_data['HH']=train_data['Dates'].apply(lambda x:int(x[11:13]))
train_data['MM']=train_data['Dates'].apply(lambda x:int(x[14:16]))
train_data['SS']=train_data['Dates'].apply(lambda x:int(x[18:21]))
train_data.drop('Dates',axis=1,inplace=True)

In [ ]:
#train_data=pd.concat([train_data,pd.get_dummies(train_data['Descript'])],axis=1)
train_data.drop('Descript',axis=1,inplace=True)

train_data=pd.concat([train_data,pd.get_dummies(train_data['DayOfWeek'])],axis=1)
train_data.drop('DayOfWeek',axis=1,inplace=True)

train_data=pd.concat([train_data,pd.get_dummies(train_data['PdDistrict'])],axis=1)
train_data.drop('PdDistrict',axis=1,inplace=True)

#train_data=pd.concat([train_data,pd.get_dummies(train_data['Resolution'])],axis=1)
train_data.drop('Resolution',axis=1,inplace=True)

train_data.drop('Address',axis=1,inplace=True)


train_data_labels=pd.get_dummies(train_data['Category'])
train_data.drop('Category',axis=1,inplace=True)
train_data.head()

In [ ]:
train_data_labels

In [ ]:
test_data['year']=test_data['Dates'].apply(lambda x:int(x[0:4]))
test_data['month']=test_data['Dates'].apply(lambda x:int(x[5:7]))
test_data['day']=test_data['Dates'].apply(lambda x:int(x[8:10]))
test_data['HH']=test_data['Dates'].apply(lambda x:int(x[11:13]))
test_data['MM']=test_data['Dates'].apply(lambda x:int(x[14:16]))
test_data['SS']=test_data['Dates'].apply(lambda x:int(x[18:21]))
test_data.drop('Dates',axis=1,inplace=True)

In [ ]:
#test_data=pd.concat([test_data,pd.get_dummies(test_data['Descript'])],axis=1)
#test_data.drop('Descript',axis=1,inplace=True)

test_data=pd.concat([test_data,pd.get_dummies(test_data['DayOfWeek'])],axis=1)
test_data.drop('DayOfWeek',axis=1,inplace=True)

test_data=pd.concat([test_data,pd.get_dummies(test_data['PdDistrict'])],axis=1)
test_data.drop('PdDistrict',axis=1,inplace=True)

#test_data=pd.concat([test_data,pd.get_dummies(test_data['Resolution'])],axis=1)
#test_data.drop('Resolution',axis=1,inplace=True)

test_data.drop('Address',axis=1,inplace=True)
test_data.head()

In [ ]:
X=train_data.to_numpy()
y=train_data_labels.to_numpy()

In [ ]:
X

In [ ]:
from tensorflow import keras
from keras.layers import Dense,Dropout
from keras.models import Sequential

In [ ]:
model=Sequential()
model.add(Dense(64,input_shape=(X.shape[1],)))
model.add(Dense(32,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(np.shape(y)[1],activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
train=model.fit(x=X,y=y,validation_split=0.2,epochs=1)

In [ ]:
Xtest=test_data.drop('Id',axis=1,inplace=False).to_numpy()
pred=model.predict(x=Xtest)
pred[0]

In [ ]:
m_reshape = np.max(pred, axis=1).reshape(-1, 1)
pred_final = np.array((pred == m_reshape), dtype='int32')
pred_final

In [ ]:
s_sub_col=list(sample_submission.columns)
print(s_sub_col)

In [ ]:
s_sub_col.remove('Id')

In [ ]:
submission = pd.DataFrame({'Id':test_data['Id']})
for i , entry in enumerate(s_sub_col):
    submission[entry] = pred_final[:,i]

In [ ]:
submission.head()

In [ ]:
submission.to_csv('../working/submission.csv', index=False)